In [ ]:
%load_ext autoreload

# Session 3: MPS sample states

In [ ]:
import numpy as np

from mps.state import *

### Simple states

We build a function that creates a simple product state in MPS form.

In [ ]:
# file: mps/state.py


def product(vectors, length=None):
    #
    # If `length` is `None`, `vectors` will be a list of complex vectors
    # representing the elements of the product state.
    #
    # If `length` is an integer, `vectors` is a single complex vector and
    # it is repeated `length` times to build a product state.
    #
    def to_tensor(v):
        v = np.array(v)
        return np.reshape(v, (1, v.size, 1))

    if length is not None:
        return MPS([to_tensor(vectors)] * length)
    else:
        return MPS(map(to_tensor, vectors))
    


### Qubit entangled states

The GHZ state is the Schrödinger cat state
$$\frac{1}{\sqrt{2}}(|0\rangle^{\otimes N} + |1\rangle^{\otimes N}).$$

In [ ]:
# file: mps/state.py


def GHZ(n):
    """Return a GHZ state with `n` qubits in MPS form."""
    a = np.zeros((2, 2, 2))
    b = a.copy()
    a[0, 0, 0] = a[0, 1, 1] = 1.0/np.sqrt(2.0)
    b[0, 0, 0] = 1.0
    b[1, 1, 1] = 1.0
    data = [a]+[b] * (n-1)
    data[0] = a[0:1, :, :]
    b = data[n-1]
    data[n-1] = (b[:, :, 1:2] + b[:, :, 0:1])
    return MPS(data)

The W state of one excitation is a delocalized spin-wave over N qubits
$$\sum \frac{1}{\sqrt{N}}\sigma^+_i |0\rangle^{\otimes N}$$

In [ ]:
# file: mps/state.py


def W(n):
    """Return a W with one excitation over `n` qubits."""
    a = np.zeros((2, 2, 2))
    a[0, 0, 0] = 1.0
    a[0, 1, 1] = 1.0/np.sqrt(n)
    a[1, 0, 1] = 1.0
    data = [a] * n
    data[0] = a[0:1, :, :]
    data[n-1] = data[n-1][:, :, 1:2]
    return MPS(data)


This is a particular case of a spin-wave
$$\sum_i \psi_i \sigma^+_i |0\rangle^{\otimes N}$$

In [ ]:
# file: mps/state.py


def wavepacket(ψ):
    #
    # Create an MPS for a spin 1/2 system with the given amplitude
    # of the excited state on each site. In other words, we create
    #
    #   \sum_i Ψ[i] σ^+ |0000...>
    #
    # The MPS is created with a single tensor: A(i,s,j)
    # The input index "i" can take two values, [0,1]. If it is '0'
    # it means we have not applied any σ^+ anywhere else, and we can
    # excite a spin here. Therefore, we have two possible values:
    #
    #   A(0,0,0) = 1.0
    #   A(0,1,1) = ψ[n] (n=given site)
    #
    # If i=1, then we cannot excite any further spin and
    #   A(1,0,1) = 1.0
    #
    # All other elements are zero. Of course, we have to impose
    # boundary conditions that the first site only has A(0,s,j)
    # and the last site only has A(i,s,1) (at least one spin has
    # been excited)
    #
    ψ = np.array(ψ)
    data = [0] * ψ.size
    for n in range(0, ψ.size):
        B = np.zeros((2, 2, 2), dtype=ψ.dtype)
        B[0, 0, 0] = B[1, 0, 1] = 1.0
        B[0, 1, 1] = ψ[n]
        data[n] = B
    data[0] = data[0][0:1, :, :]
    data[-1] = data[-1][:, :, 1:]
    return MPS(data)

Create the graph states for qubits in 1D.

In [ ]:
# file: mps/state.py


def graph(n, mps=True):
    """Create a one-dimensional graph state of `n` qubits."""
    # Choose entangled pair state as : |00>+|11>
    # Apply Hadamard H on the left virtual spins (which are the right spins of the entangled bond pairs)
    H = np.array([[1, 1], [1, -1]])
    # which gives |0>x(|0>+|1>)+|1>x(|0>-|1>) = |00>+|01>+|10>-|11>
    # Project as  |0><00| + |1><11|
    # We get the following MPS projectors:
    A0 = np.dot(np.array([[1, 0], [0, 0]]), H)
    A1 = np.dot(np.array([[0, 0], [0, 1]]), H)
    AA = np.array([A0, A1])
    AA = np.swapaxes(AA, 0, 1)
    data = [AA]*n
    data[0] = np.dot(np.array([[[1, 0], [0, 1]]]), H)
    data[-1] = np.swapaxes(np.array([[[1, 0], [0, 1]]]), 0, 2) / np.sqrt(2**n)

    return MPS(data)

## Qutrit states

In [ ]:
# file: mps/state.py

# open boundary conditions
# free virtual spins at both ends are taken to be zero


def AKLT(n, mps=True):
    """Return an AKL state with `n` spin-1 particles."""
    # Choose entangled pair state as : |00>+|11>
    # Apply i * Pauli Y matrix on the left virtual spins (which are the right spins of the entangled bond pairs)
    iY = np.array([[0, 1], [-1, 0]])
    # which gives -|01>+|10>
    # Project as  |-1><00| +|0> (<01|+ <10|)/ \sqrt(2)+ |1><11|
    # We get the following MPS projectors:
    A0 = np.dot(np.array([[1, 0], [0, 0]]), iY)
    A1 = np.dot(np.array([[0, 1], [1, 0]]), iY)
    A2 = np.dot(np.array([[0, 0], [0, 1]]), iY)

    AA = np.array([A0, A1, A2]) / np.sqrt(2)
    AA = np.swapaxes(AA, 0, 1)
    data = [AA]*n
    data[-1] = np.array([[[1, 0], [0, 1], [0, 0]]])
    data[0] = np.array(np.einsum('ijk,kl->ijl',
                                 data[-1], iY))/np.sqrt(2)
    data[-1] = np.swapaxes(data[-1], 0, 2)

    return MPS(data)


## Generic

In [ ]:
# file: mps/state.py


def random(d, N, D=1):
    """Create a random state with 'N' elements of dimension 'd' and bond
    dimension 'D'."""
    mps = [1]*N
    DR = 1
    for i in range(N):
        DL = DR
        if N > 60 and i != N-1:
            DR = D
        else:
            DR = np.min([DR*d, D, d**(N-i-1)])
        mps[i] = np.random.rand(DL, d, DR)
    return MPS(mps)

In [ ]:
def gaussian(n, x0, w0, k0, mps=True):
    #
    # Return a W state with `n` components in MPS form or
    # in vector form
    #
    xx = np.arange(n, dtype=complex)
    coefs = np.exp(-(xx-x0)**2 / w0**2 + 1j * k0*xx, dtype=complex)
    return wavepacket(coefs / np.linalg.norm(coefs))

----

# Tests

In [ ]:
# file: mps/test/test_sample_states.py

import unittest


class TestSampleStates(unittest.TestCase):

    def test_product_state(self):
        a = np.array([1.0, 7.0])
        b = np.array([0.0, 1.0, 3.0])
        c = np.array([3.0, 5.0])

        # Test a product state MPS of size 3
        state1 = product(a, length=3)
        tensor1 = np.reshape(a, (1, 2, 1))
        #
        # Test whether the MPS has the right size and dimension
        self.assertEqual(state1.size, 3)
        self.assertEqual(state1.dimension(), 8)
        #
        # Verify that it has the same data as input
        self.assertTrue(np.array_equal(state1[0], tensor1))
        self.assertTrue(np.array_equal(state1[1], tensor1))
        self.assertTrue(np.array_equal(state1[2], tensor1))
        #
        # Verify that they produce the same wavefunction as directly
        # creating the vector
        state1ψ = np.kron(a, np.kron(a, a))
        self.assertTrue(np.array_equal(state1.tovector(), state1ψ))

        # Test a product state with different physical dimensions on
        # even and odd sites.
        state2 = product([a, b, c])
        tensor2a = np.reshape(a, (1, 2, 1))
        tensor2b = np.reshape(b, (1, 3, 1))
        tensor2c = np.reshape(c, (1, 2, 1))
        #
        # Test whether the MPS has the right size and dimension
        self.assertEqual(state2.size, 3)
        self.assertEqual(state2.dimension(), 2*3*2)
        #
        # Verify that it has the same data as input
        self.assertTrue(np.array_equal(state2[0], tensor2a))
        self.assertTrue(np.array_equal(state2[1], tensor2b))
        self.assertTrue(np.array_equal(state2[2], tensor2c))
        #
        # Verify that they produce the same wavefunction as directly
        # creating the vector
        state2ψ = np.kron(a, np.kron(b, c))
        self.assertTrue(np.array_equal(state2.tovector(), state2ψ))

    def test_GHZ(self):
        ghz1 = np.array([1.0, 1.0])/np.sqrt(2.0)
        mps1 = GHZ(1)
        self.assertTrue(np.array_equal(mps1.tovector(), ghz1))

        ghz2 = np.array([1.0, 0.0, 0.0, 1.0])/np.sqrt(2.0)
        mps2 = GHZ(2)
        self.assertTrue(np.array_equal(mps2.tovector(), ghz2))

        ghz3 = np.array([1.0, 0, 0, 0, 0, 0, 0, 1.0])/np.sqrt(2.0)
        mps3 = GHZ(3)
        self.assertTrue(np.array_equal(mps3.tovector(), ghz3))

        for i in range(1, 2):
            Ψ = GHZ(i)
            self.assertEqual(Ψ.size, i)
            self.assertEqual(Ψ.dimension(), 2**i)

    def test_W(self):
        W1 = np.array([0, 1.0])
        mps1 = W(1)
        self.assertTrue(np.array_equal(mps1.tovector(), W1))

        W2 = np.array([0, 1, 1, 0])/np.sqrt(2.0)
        mps2 = W(2)
        self.assertTrue(np.array_equal(mps2.tovector(), W2))

        W3 = np.array([0, 1, 1, 0, 1, 0, 0, 0])/np.sqrt(3.0)
        mps3 = W(3)
        self.assertTrue(np.array_equal(mps3.tovector(), W3))

        for i in range(1, 2):
            Ψ = W(i)
            self.assertEqual(Ψ.size, i)
            self.assertEqual(Ψ.dimension(), 2**i)

    def test_AKLT(self):
        AKLT2 = np.zeros(3**2)
        AKLT2[1] = 1
        AKLT2[3] = -1
        AKLT2 = AKLT2/np.sqrt(2)
        self.assertTrue(np.array_equal(AKLT(2).tovector(), AKLT2))

        AKLT3 = np.zeros(3**3)
        AKLT3[4] = 1
        AKLT3[6] = -1
        AKLT3[10] = -1
        AKLT3[12] = 1
        AKLT3 = AKLT3/(np.sqrt(2)**2)
        self.assertTrue(np.array_equal(AKLT(3).tovector(), AKLT3))

        for i in range(1, 5):
            Ψ = AKLT(i)
            self.assertEqual(Ψ.size, i)
            self.assertEqual(Ψ.dimension(), 3**i)

    def test_graph(self):
        GR = np.ones(2**2)/np.sqrt(2**2)
        GR[-1] = -GR[-1]
        self.assertTrue(np.array_equal(graph(2).tovector(), GR))

        GR = np.ones(2**3)/np.sqrt(2**3)
        GR[3] = -GR[3]
        GR[-2] = -GR[-2]
        self.assertTrue(np.array_equal(graph(3).tovector(), GR))

        for i in range(1, 2):
            Ψ = W(i)
            self.assertEqual(Ψ.size, i)
            self.assertEqual(Ψ.dimension(), 2**i)

In [ ]:
%autoreload
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestSampleStates'])
unittest.TextTestRunner(verbosity=2).run(suite1);